# Demosaickin<font color='green'>g</font> with the help of li<font color='blue'>b</font><font color='red'>r</font>ary functions
For future reference (and for all <font color='red'>R</font><font color='green'>G</font><font color='blue'>B</font> channels) - but with **almost** no educational charm **what.so.ever**...

**Almost:** Illustration of application of superposition:
$$
\begin{bmatrix}
\color{red}{R} & \color{green}{G_I}\\
\color{green}{G_{II}} & \color{blue}{B}
\end{bmatrix}
=
\begin{bmatrix}
\color{red}{R} & 0\\
0 & 0
\end{bmatrix}
+
\underset{\color{green}{G}}{\underbrace{\left[
\begin{bmatrix}
0 & \color{green}{G_I}\\
0 & 0
\end{bmatrix}
+
\begin{bmatrix}
0 & 0\\
\color{green}{G_{II}} & 0
\end{bmatrix}
\right]}}
+
\begin{bmatrix}
0 & 0\\
0 & \color{blue}{B}
\end{bmatrix}
$$

In [ ]:
import cv2; import numpy as np; import auxiliary as aux

scheme = cv2.INTER_LINEAR_EXACT
img = cv2.resize(cv2.cvtColor(cv2.imread('./images/GrassHopper.png'), cv2.COLOR_BGR2RGB),
                 (128, 128),
                 interpolation = scheme)

## Moisaicking (Bayer CFA)
N, M, _ = img.shape
raws  = img[0:N:2, 0:M:2, 0], img[1:N:2, 1:M:2, 2], img[1:N:2, 0:M:2, 1], img[0:N:2, 1:M:2, 1]

## De-mosaicking (de-Bayering by interpolation)
# Note how we use superposition to restore the green channel
R, B, GI, GII = [cv2.resize(channel_raw, (M, N), interpolation = scheme) for channel_raw in raws]
G = GI//2 + GII//2

images = ('Original', 'R', 'B', '1st G', '2nd G', 'G')
aux.displayImages((img, R, B, GI, GII, G), images, grid = False)

rgb = np.dstack((R, G, B)); cc = ('red', 'green', 'blue')
aux.displayChannels((img, rgb), cc)

images = ('Original', 'Demo\'ed', 'Little diff\'s') # So to speak... https://youtu.be/ab7eVVG3I8s?t=43
aux.displayImages((img, rgb, rgb - img), images, grid = False)

## A **s[ligh|ignifican]tly** more re-educational version...
Here we re-use (albeit for the **<font color='red'>R</font> channel** only...) our own (and a bit majestic) interpolating routine!

In [ ]:
from interpolation import Π, Λ, ϕ, ξ, interpolate as ΣΣ
from auxiliary import displayImages as DI
from matplotlib.colors import LinearSegmentedColormap as lscm

# Linear interpolation
ψ = ϕ; name = ψ.__name__
reds = lscm.from_list('_', ['black', 'red'])

red = np.zeros((N, N))
# Rows first...
for m in range(M >> 1):
   red[m, ...] = np.matrix(ΣΣ(raws[0][m, ...], N, φ = ψ)).flat    # matrix?
DI((img, red), ('Original', f'{name}-scaled rows'), cmp = reds, grid = False)
# ...then columns
for n in range(N):
   red[:, n] = np.array(ΣΣ(red[:M >> 1, n], N, φ = ψ)).flat       # array? whatchamacallit!
DI((img, red), ('Original', f'{name}-scaled rows & columns'), cmp = reds, grid = False)